In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
train_id_df = pd.read_csv('data/train_identity.csv')
test_id_df = pd.read_csv('data/test_identity.csv')

train_transaction_df = pd.read_csv('data/train_transaction.csv')
test_transaction_df = pd.read_csv('data/test_transaction.csv')



In [3]:
test_transaction_df.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train_transaction_y = train_transaction_df.isFraud
train_transaction_df.drop('isFraud', axis=1, inplace=True)

In [5]:
#train_id_df.head()

In [6]:
#train_id_df.describe().transpose()

In [7]:
# show drop columns 

missing_threshold = 0.8  #percentage of values missing to consider drop

# missing_id_columns = train_id_df.isnull().sum()/len(train_id_df) 
# drop_columns =  (missing_id_columns[missing_id_columns > missing_threshold]).axes[0]
# train_id_df.drop(drop_columns, axis=1, inplace=True)
# test_id_df.drop(drop_columns, axis=1, inplace=True)
# print(drop_columns)

missing_transaction_columns = train_transaction_df.isnull().sum()/len(train_transaction_df) 
drop_columns =  (missing_transaction_columns[missing_transaction_columns > missing_threshold]).axes[0]
train_transaction_df.drop(drop_columns, axis=1, inplace=True)
test_transaction_df.drop(drop_columns, axis=1, inplace=True)
print(drop_columns)

Index(['dist2', 'D6', 'D7', 'D8', 'D9', 'D12', 'D13', 'D14', 'V138', 'V139',
       'V140', 'V141', 'V142', 'V143', 'V144', 'V145', 'V146', 'V147', 'V148',
       'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V156', 'V157',
       'V158', 'V159', 'V160', 'V161', 'V162', 'V163', 'V164', 'V165', 'V166',
       'V322', 'V323', 'V324', 'V325', 'V326', 'V327', 'V328', 'V329', 'V330',
       'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339'],
      dtype='object')


In [8]:
train_transaction_df.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0
1,2987001,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987002,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987003,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0
4,2987004,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
test_transaction_df.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,77.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,263.0,0.0
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,0.000000,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:

column_list = train_transaction_df.columns
column_list[column_list != 'TransactionID']

Index(['TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2',
       'card3', 'card4', 'card5', 'card6', 'addr1',
       ...
       'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320',
       'V321'],
      dtype='object', length=337)

In [11]:
# impute missing values
my_imputer = SimpleImputer(strategy='median')
column_list = train_transaction_df.columns

train_transaction_df[column_list[column_list != 'TransactionID']] = pd.DataFrame(my_imputer.fit_transform(train_transaction_df[column_list[column_list != 'TransactionID']]))
#train_transaction_df.columns = column_list

test_transaction_df[column_list[column_list != 'TransactionID']] = pd.DataFrame(my_imputer.transform(test_transaction_df[column_list[column_list != 'TransactionID']]))
#test_transaction_df.columns = column_list

test_transaction_df.head()

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [ ]:
# Scale values
my_scaler = StandardScaler()

train_transaction_df[column_list[column_list != 'TransactionID']] = pd.DataFrame(my_scaler.fit_transform(train_transaction_df[column_list[column_list != 'TransactionID']]))

test_transaction_df[column_list[column_list != 'TransactionID']] = pd.DataFrame(my_scaler.transform(test_transaction_df[column_list[column_list != 'TransactionID']]))



In [ ]:

s = (train_transaction_df.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transformer(train_transaction_df[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(test_transaction_df[object_cols]))


# One-hot encoding removed index; put it back
OH_cols_train.index = train_transaction_df.index
OH_cols_test.index = test_transaction_df.index


# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_transaction_df.drop(object_cols, axis=1)
num_X_test = test_transaction_df.drop(object_cols, axis=1)


# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

train_transaction_df = OH_X_train
test_transaction_df = OH_X_test

test_transaction_df.head()

In [ ]:
test_transaction_df.head()

In [ ]:
missing_transaction_columns = train_transaction_df.isnull().sum()/len(train_transaction_df) 
missing_transaction_columns

In [ ]:
# #convert low cardinality categorical columns and drop high cardinality categorical colums

# cardinality_threshold =  10

# cat_id_cols = [cname for cname in train_id_df.columns if train_id_df[cname].dtype == 'object']
# low_id_cat_cols = [cname for cname in cat_id_cols if train_id_df[cname].nunique() < cardinality_threshold]
# high_id_cat_cols = list(set(cat_id_cols) - set(low_id_cat_cols))

# print(high_id_cat_cols)

# train_id_df.drop(high_id_cat_cols, axis=1, inplace=True)
# test_id_df.drop(high_id_cat_cols, axis=1, inplace=True)


# cat_cols = [cname for cname in train_transaction_df.columns if train_transaction_df[cname].dtype == 'object']
# low_cat_cols = [cname for cname in cat_cols if train_transaction_df[cname].nunique() < cardinality_threshold]
# high_cat_cols = list(set(cat_cols) - set(low_cat_cols))


# print(high_cat_cols)

# train_transaction_df.drop(high_cat_cols, axis=1, inplace=True)
# test_transaction_df.drop(high_cat_cols, axis=1, inplace=True)


In [ ]:
# # One hot encoder
# oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
# oh_cols_train = pd.DataFrame(oh_encoder.fit_transform(train_transaction_df[low_cat_cols]))



In [ ]:
# Exclude high correlation
transaction_corr = train_transaction_df.corr().abs()

train_transaction_df_original = train_transaction_df

In [ ]:
high_corr_threshold = 0.80
upper_transaction_corr = transaction_corr.where(np.triu(np.ones(transaction_corr.shape), k=1).astype(np.bool))
to_drop = [col for col in upper_transaction_corr.columns if any(upper_transaction_corr[col] > high_corr_threshold)]


In [ ]:
train_transaction_df = train_transaction_df_original.drop(to_drop, axis=1)

In [ ]:
train_ratio = 0.80

train_ids = train_transaction_df.TransactionID
y = train_transaction_y
X = train_transaction_df

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=train_ratio, test_size=1-train_ratio,
                                                      random_state=0)
X_train_ids = X_train['TransactionID']
X_valid_ids = X_valid['TransactionID']

X_train.drop('TransactionID', axis=1, inplace=True)
X_valid.drop('TransactionID', axis=1, inplace=True)

In [ ]:
param = {'num_leaves': 1011, 'objective': 'binary'}
param['metric'] = 'auc'


train_data_lgb = lgb.Dataset(X_train, label=y_train)
valid_data_lgb = lgb.Dataset(X_valid, label=y_valid)

model = lgb.train(param, train_data_lgb, num_boost_round=800, early_stopping_rounds=100, valid_sets=[valid_data_lgb])
preds = model.predict(X_valid)
mean_absolute_error(y_valid, preds)
#model_lgbm.fit(X_train, y_train)
#preds = model_lgbm.predict(X_valid)
#mean_absolute_error(y_valid, preds)

In [ ]:
# preds_df = pd.DataFrame(preds)
# y_valid_df = pd.DataFrame(y_valid)
# y_valid_df.reset_index(inplace=True)
# y_valid_df.drop('index', axis=1, inplace=True)

# result =pd.concat([preds_df, y_valid_df], axis=1)

# # Check how many fraud cases have predicted probability > 0.5
# print(len(X_valid.columns))
# print(len(result[(result.isFraud == 1) & (result[0] > 0.5)]))
# print(len(result[(result.isFraud == 1) & (result[0] <= 0.5)]))

# 132
# 3031
# 2685

In [ ]:
preds_df = pd.DataFrame(preds)
y_valid_df = pd.DataFrame(y_valid)
y_valid_df.reset_index(inplace=True)
y_valid_df.drop('index', axis=1, inplace=True)

result =pd.concat([preds_df, y_valid_df], axis=1)

# Check how many fraud cases have predicted probability > 0.5
print(len(X_valid.columns))
print(len(result[(result.isFraud == 1) & (result[0] > 0.5)]))
print(len(result[(result.isFraud == 1) & (result[0] <= 0.5)]))

In [ ]:
test_ids = test_transaction_df.TransactionID
X_test = test_transaction_df[X.columns]
X_test.drop('TransactionID', axis=1, inplace=True)

X_test.shape

In [ ]:
test_pred_df = pd.DataFrame(model.predict(X_test))
test_pred_df.head()

In [ ]:
pd.DataFrame(test_ids)
output_df = pd.concat([pd.DataFrame(test_ids), test_pred_df[0]], axis=1)
output_df.head()

In [ ]:
output_df.columns = ['TransactionID', 'isFraud']
output_df['TransactionID'] = output_df['TransactionID'].astype(int)
output_df.to_csv('submission_lgbm.csv', index=False)

In [ ]:
test_transaction_df.head()
